# Benchmark of 4D phase space reconstruction algorithms 

In [ ]:
import sys
import importlib
import numpy as np
from skimage import filters
from matplotlib import pyplot as plt
import proplot as pplt
from tqdm.notebook import tqdm
from tqdm.notebook import trange

import reconstruct as rec

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['grid.alpha'] = 0.04

## Setup 

In [ ]:
X = np.load('data/coords_SNS_elliptical_painting_300turns.npy')
X = X[:, :4]
X = X - np.mean(X, axis=0)

Compute the 2D Twiss parameters; normalize $x$-$x'$ and $y$-$y'$.

In [ ]:
import tools.analysis as ba
from tools import ap_utils
from tools import utils

In [ ]:
Sigma = np.cov(X.T)
alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
print('Statistical Twiss parameters:')
print('  alpha_x = {}'.format(alpha_x))
print('  alpha_y = {}'.format(alpha_y))
print('  beta_x = {}'.format(beta_x))
print('  beta_y = {}'.format(beta_y))

# Define the lattice Twiss parameters. (If `dalpha` and `dbeta` are nonzero, 
# the lattice Twiss parameters will be different than the statistical Twiss
# parameters. 
np.random.seed(17)
dalpha = 0.0
dbeta = 0.0 * max(beta_x, beta_y)
alpha_x += np.random.uniform(-dalpha, dalpha)
alpha_y += np.random.uniform(-dalpha, dalpha)
beta_x += np.random.uniform(-dbeta, dbeta)
beta_y += np.random.uniform(-dbeta, dbeta)
print('Lattice Twiss parameters:')
print('  alpha_x = {}'.format(alpha_x))
print('  alpha_y = {}'.format(alpha_y))
print('  beta_x = {}'.format(beta_x))
print('  beta_y = {}'.format(beta_y))

V = ap_utils.V_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
Vinv = np.linalg.inv(V)
Xn = utils.apply(Vinv, X)

Bin the normalized coordinates at the reconstruction location.

In [ ]:
n_bins = 20

In [ ]:
import tools.plotting as mplt

In [ ]:
axes = mplt.corner(Xn, figsize=(6, 6), bins=n_bins, autolim_kws=dict(sigma=3.5))
rec_limits = [ax.get_xlim() for ax in axes[-1, :]]

In [ ]:
Z_true, rec_edges = np.histogramdd(Xn, n_bins, rec_limits, density=True)
rec_centers = [rec.get_bin_centers(_edges) for _edges in rec_edges]
bin_volume = rec.get_bin_volume(rec_limits, n_bins)

Transport the distribution to the screen. Since the horizontal and vertical optics are varied independently, we can transport $x$-$x'$ and $y$-$y'$ independently. Assume the phase advances are evenly spaced over 180 degrees. The Twiss parameters at the screen are randomly chosen in a realistic range.

In [ ]:
def make_tmats(phase_advances, betas, alphas, Vinv):
    """Return list of 2x2 transfer matrices."""
    tmats = []
    for phase_adv, beta, alpha in zip(phase_advances, betas, alphas):
        P = utils.rotation_matrix(phase_adv)
        V_screen = ap_utils.V_matrix_2x2(alpha, beta)
        tmats.append(np.linalg.multi_dot([V_screen, P, Vinv]))
    return tmats

In [ ]:
K = 12  # number of horizontal optics settings
L = 12  # number of vertical optics settings

phase_adv_x = np.linspace(0.0, np.pi, K, endpoint=False)
betas_x = np.random.uniform(1.0, 1.0, size=K)
alphas_x = np.random.uniform(0.0, 0.0, size=K)

phase_adv_y = np.linspace(0.0, np.pi, L, endpoint=False)
betas_y = np.random.uniform(1.0, 1.0, size=L)
alphas_y = np.random.uniform(0.0, 0.0, size=L)

tmats_x = make_tmats(phase_adv_x, betas_x, alphas_x, Vinv[:2, :2])
tmats_y = make_tmats(phase_adv_y, betas_y, alphas_y, Vinv[2:, 2:])
xx_list = [utils.apply(Mx, X[:, :2])[:, 0] for Mx in tqdm(tmats_x)]
yy_list = [utils.apply(My, X[:, 2:])[:, 0] for My in tqdm(tmats_y)]

Bin the coordinates on the screen.

In [ ]:
# xmax_screen = ymax_screen = 75.0 # [mm]
# meas_edges = [
#     np.linspace(-xmax_screen, xmax_screen, n_bins + 1),
#     np.linspace(-xmax_screen, xmax_screen, n_bins + 1),
# ]
meas_edges = [rec_edges[0], rec_edges[1]]
meas_centers = [rec.get_bin_centers(_edges) for _edges in meas_edges]

In [ ]:
x_list, xp_list, y_list, yp_list = [], [], [], []
for Mx in tqdm(tmats_x):
    x, xp = utils.apply(Mx, X[:, :2]).T
    x_list.append(x)
    xp_list.append(xp)
for My in tqdm(tmats_y):
    y, yp = utils.apply(My, X[:, 2:]).T
    y_list.append(y)
    yp_list.append(yp)

Sigmas = []
for (x, xp) in tqdm(zip(x_list, xp_list)):
    for (y, yp) in zip(y_list, yp_list):
        Y = np.vstack([x, xp, y, yp]).T
        Sigmas.append(np.cov(Y.T))

In [ ]:
S = np.zeros((n_bins, n_bins, K, L))
for k, xx in enumerate(tqdm(xx_list)):
    for l, yy in enumerate(yy_list):
        S[:, :, k, l], _, _ = np.histogram2d(xx, yy, meas_edges)

In [ ]:
# fig, axes = pplt.subplots(nrows=3, ncols=5, figwidth=8.5, space=0.25)
# for k, ax in enumerate(axes):
#     ax.pcolormesh(S[:, :, k, 0].T, cmap='mono_r', ec='None')
#     ax.annotate(r'$\mu_x = {:.0f}\degree$'.format(np.degrees(phase_adv_x[k])), 
#                 xy=(0.01, 0.93), xycoords='axes fraction', color='white', fontsize='small')
# axes.format(xlabel=r"$x$", ylabel=r"$y$", xticks=[], yticks=[])
# plt.savefig('_output/tomo_sim_target_scan.png', dpi=500)
# plt.show()

In [ ]:
fig, axes = pplt.subplots(nrows=K, ncols=L, figwidth=10, space=0.175)
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        ax = axes[i, j]
        ax.pcolormesh(S[:, :, i, j].T, cmap='mono_r', ec='None')
axes.format(
    xticks=[], yticks=[], 
#     xlabel='x', ylabel='y',
)
# plt.savefig('_output/tomo_sim_target_scan_full.png', dpi=500)
plt.show()

## Hock's method

We'll reconstruct in normalized phase space.

In [ ]:
tmats_x_n = [np.matmul(Mx, V[:2, :2]) for Mx in tmats_x]
tmats_y_n = [np.matmul(My, V[2:, 2:]) for My in tmats_y]

In [ ]:
Z_hock = rec.hock4D(S, meas_centers, [rec_centers[0], rec_centers[2]], 
                    tmats_x_n, tmats_y_n, method='SART', iterations=3)
Z_hock = rec.process(Z_hock, keep_positive=True, density=True, limits=rec_limits)

In [ ]:
print('min(Z) = {}'.format(np.min(Z_hock)))
print('max(Z) = {}'.format(np.max(Z_hock)))
print('sum(Z) * bin_volume = {}'.format(np.sum(Z_hock) * bin_volume))
print()
print('min(Z_true) = {}'.format(np.min(Z_true)))
print('max(Z_true) = {}'.format(np.max(Z_true)))
print('sum(Z_true) * bin_volume = {}'.format(np.sum(Z_true) * bin_volume))

In [ ]:
f_hock = Z_hock
f_true = Z_true

In [ ]:
def plot_compare_1D(f1, f2):
    fig, axes = pplt.subplots(ncols=6, nrows=3, figwidth=8)
    inds = [(0, 2), (0, 1), (2, 3), (0, 3), (2, 1), (1, 3)]
    dims = ["x", "x'", "y", "y'"]
    for j, ind in enumerate(inds):
        x = rec_centers[ind[0]]
        y = rec_centers[ind[1]]
        H1 = rec.project(f1, ind)
        H2 = rec.project(f2, ind)
        Hdiff = H1 - H2
        for ax, H in zip(axes[:, j], [H1, H2, Hdiff]):
            ax.pcolormesh(x, y, H.T)
        axes[0, j].format(title=f'{dims[ind[0]]}-{dims[ind[1]]}')
    plt.show()

In [ ]:
axes = plot_compare_1D(f_hock, f_true)
# plt.savefig('_output/tomo_sim_rec_hock_proj_1D.png', dpi=500)

Transform to unnormalized phase space.

Study 2D reconstruction as function of number of angles and of the angular range.

In [ ]:
# from skimage.transform import radon, iradon_sart, iradon

# i1, i2 = 2, 3
# bins = 50
# z_true, _, _ = np.histogram2d(Xn[:, i1], Xn[:, i2], bins=bins, 
#                               range=[rec_limits[i1], rec_limits[i2]], density=True)

# n_proj_list = np.arange(3, 30)
# max_angles = np.linspace(5., 180., 50, endpoint=False)
# errors = np.zeros((len(n_proj_list), len(max_angles)))    
# for i in trange(errors.shape[0]):
#     for j in range(errors.shape[1]):
#         angles = np.linspace(0., max_angles[j], n_proj_list[i])
#         projections = radon(z_true, theta=angles, circle=True)
#         z = iradon_sart(projections, theta=angles)
#         for _ in range(1):
#             z = iradon_sart(projections, theta=angles, image=z)
#         z = rec.process(z, keep_positive=True, density=True, limits=[rec_limits[i1], rec_limits[i2]])
#         # if i == 0:
#         #     fig, axes = pplt.subplots(ncols=2)
#         #     axes[0].pcolormesh(xedges, yedges, Z_true.T, cmap='viridis',)
#         #     axes[1].pcolormesh(xedges, yedges, Z.T, cmap='viridis',)
#         #     plt.show()
#         errors[i, j] = np.sqrt(np.sum((z - z_true)**2))
        
# fig, ax = pplt.subplots()
# ax.pcolormesh(
#     n_proj_list, max_angles, errors.T, cmap=pplt.Colormap('mono'), 
#     # vmin=0,
#     colorbar=True, discrete=False,
#     colorbar_kw=dict(label=r'$\Vert Z - Z_{true} \Vert$', width=0.1, space=1.0)
# )
# ax.format(yformatter='deg', ylabel='Angular range', xlabel='Number of projections')
# plt.savefig('_output/tomo_sim_art2d.png', dpi=350)
# plt.show()

## 4D ART 

The maximum reconstruction grid resolution for this method is approximately 50.

In [ ]:
n_bins = 20
Z_true2, rec_edges = np.histogramdd(Xn, n_bins, rec_limits, density=True)
rec_centers = [rec.get_bin_centers(_edges) for _edges in rec_edges]
bin_volume = rec.get_bin_volume(rec_limits, n_bins)

In [ ]:
meas_edges = [rec_edges[0], rec_edges[2]]
meas_centers = [rec.get_bin_centers(_edges) for _edges in meas_edges]

In [ ]:
K = 8 # number of horizontal optics settings
L = 8 # number of vertical optics settings

phase_adv_x = np.linspace(0.0, np.pi, K, endpoint=False)
betas_x = np.random.uniform(1.0, 1.0, size=K)
alphas_x = np.random.uniform(0.0, 0.0, size=K)

phase_adv_y = np.linspace(0.0, np.pi, L, endpoint=False)
betas_y = np.random.uniform(1.0, 1.0, size=L)
alphas_y = np.random.uniform(0.0, 0.0, size=L)

tmats_x = make_tmats(phase_adv_x, betas_x, alphas_x, Vinv[:2, :2])
tmats_y = make_tmats(phase_adv_y, betas_y, alphas_y, Vinv[2:, 2:])
xx_list = [utils.apply(Mx, X[:, :2])[:, 0] for Mx in tmats_x]
yy_list = [utils.apply(My, X[:, 2:])[:, 0] for My in tmats_y]

In [ ]:
tmats_x_n = [np.matmul(Mx, V[:2, :2]) for Mx in tmats_x]
tmats_y_n = [np.matmul(My, V[2:, 2:]) for My in tmats_y]
projections, tmats_n = [], []
for xx, Mx in zip(xx_list, tmats_x_n):
    for yy, My in zip(yy_list, tmats_y_n):
        projection, _, _ = np.histogram2d(xx, yy, meas_edges)
        projections.append(projection)
        M = np.zeros((4, 4))
        M[:2, :2] = Mx
        M[2:, 2:] = My
        tmats_n.append(M)

In [ ]:
Z_art = rec.art4D(projections, tmats_n, rec_centers, meas_centers)
Z_art = rec.process(Z_art, keep_positive=True, density=True, limits=rec_limits)

In [ ]:
axes = plot_compare_1D(Z_art, Z_true2)

In [ ]:
from tools.plotting2 import interactive_proj2d

In [ ]:
interactive_proj2d(Z_art / Z_art.max(), dims=["x", "xp", "y", "yp"], coords=rec_centers)

## Particle-based

In [ ]:
importlib.reload(rec)

In [ ]:
projections = []
tmats = []
for k in np.arange(0, S.shape[2]):
    for l in np.arange(0, S.shape[3]):
        projections.append(S[:, :, k, l])
        M = np.zeros((4, 4))
        M[:2, :2] = tmats_x[k]
        M[2:, 2:] = tmats_y[l]
        tmats.append(M)
Z_pic, _proj = rec.pic4D(projections, tmats, rec_centers, screen_edges, max_iters=15)

In [ ]:
compare(Z_pic, Z_true)

In [ ]:
Z = np.load('_output/Z_hock.npy')
Z_true = np.load('_output/Z_true.npy')

space = 0.5
indices = [(0, 1), (2, 3), (0, 2), (0, 3), (2, 1), (1, 3)]
labels = ["x", "x'", "y", "y'"]
plot_kws = dict(cmap='mono_r', ec='None')
fig, axes = pplt.subplots(nrows=6, ncols=3, figsize=(5.0, 8.5),
                          sharex=False, sharey=False, space=space)
for row, (i, j) in enumerate(indices):
    _Z_true = rec.project(Z_true, [i, j])
    _Z = rec.project(Z, [i, j])
    axes[row, 0].pcolormesh(_Z_true.T, **plot_kws)
    axes[row, 1].pcolormesh(_Z.T, **plot_kws)
    axes[row, 2].pcolormesh((_Z - _Z_true).T, 
                            diverging=True, cmap='Vlag',
                            colorbar=True, ec='None',
                            colorbar_kw=dict(width=0.075, space=space, ticklabelsize='small')
                            )
    axes[row, 0].annotate('{}-{}'.format(labels[i], labels[j]),
                           xy=(0.02, 0.92), xycoords='axes fraction', color='white', 
                           fontsize='medium')
for ax, title in zip(axes[0, :], ['True', 'Reconstructed', 'Error']):
    ax.set_title(title)
axes.format(xticks=[], yticks=[])
plt.savefig('_output/tomo_sim_rec_hock_proj_2D_ver.png', dpi=500)
plt.show()